# 18 - Binary Files

---

"Binary files" is the term used to refer to all files that are not text files. Executable programs are binary files, as are image files, movies, word processor documents, and many other file types. It is not common to use Python to process binary files (usually binary files are not handled by general-purpose programming languages, but by special-purpose programs), but it is possible. This chapter will explain how to deal with binary files.  

---

## Opening and closing binary files

The handling of binary files is quite similar to the handling of text files. You have to `open()` a file when you want to access its contents, `close()` it when you are finished, `read()` from the file and `write()` to the file.

When you open a binary file, you have to indicate to Python that you want to handle this file in "binary mode". You do this by adding a letter "`b`" to the mode argument. For instance, to open a file in "binary read" mode, the mode argument should be "`rb`". You can also open a file both for reading and writing; reading and writing you indicate with mode "`r+`", so reading and writing in binary mode is "`r+b`" (while it is also possible to open text files in "`r+`" mode, I did not indicate it in Chapter 17, as it seldom makes sense to open text files in this mode). Just as with text files, if you open a binary file in write-mode, with "`wb`", the file gets emptied. The mode "`w+b`" will open a file for both reading and writing, but also empties the file to start with.

When you open a file for both reading and writing, if the file pointer is not at the end of the file, when you write you actually *over*write.

You can open any file in binary mode, even text files. However, when you open text files in binary mode, you treat them like binary files, which means that Python does not do the automatic conversion of newline characters.

Closing a binary file is no different from closing a text file.

In [ ]:
fp = open( "pc_rose.txt", "rb" )
fp.close()

Note: The code above has no output -- if it does have output, that is a runtime error, meaning that `pc_rose.txt` is not available.

---

## Reading a binary file

As binary files do not know the concept of "lines", the only way to read from a binary file is to use the `read()` method. If you use `read()` without argument, it reads the whole file (starting at the file pointer). If you give the method an integer as parameter, that integer indicates the number of bytes that are read from the file (starting at the file pointer, and reading at maximum until the end of the file). 

A "byte", if you do not know, is an 8-bit character, i.e., a number between zero and 255, which is stored in the smallest possible memory unit that a computer supports. The regular characters on a keyboard are each stored in a single byte, and the characters in a string are also each a byte, though limited to a specific range of numbers.

### Byte strings

Here we enter one of the more obscure parts of the Python language. When you read from a binary file, the `read()` method does not return a regular string -- it returns a "byte string". There are some subtle differences between regular strings and byte strings. To show you these differences, I first have to tell you that you can indicate that a string is a byte string by placing a letter `b` in front of it. So `"Hello, world!"` is a string, while `b"Hello, world!"` is a byte string.

In [ ]:
hw1 = "Hello, world!"
hw2 = b"Hello, world!"

print( hw1 )
print( hw2 )

The difference between a string and a byte string is that a byte string can contain characters that a string cannot. For instance, if you remember the discussion on the ASCII table, you may recall that each character has a number associated with it. You saw, for instance, that "A" has the number 65, and the space has the number 32. The space was the lowest numbered character that I showed, and you might wonder which characters are associated with numbers 0 to 31. The answer is: these are control codes, and are not legal characters that you can put in a string. You can *try* to put them in a string using an "escape code": the escape sequence `\x` can be followed by a two-character hexadecimal code that represents the character with the specified number. For example, the hexadecimal code for a space is `\x20`, i.e., `"Hello, world!"` is the same as `"Hello,\x20world!"` (this was discussed in the chapter on strings).

In [ ]:
hw1 = "Hello,\x20world!"
print( hw1 )

But what if you try to put illegal characters in a string that way? They are ignored:

In [ ]:
hw1 = "Hello,\x00\x01\x02world!"
print( hw1 )

The problem is that such characters can occur in binary files, so you must be able to read them from binary files. Since byte strings *can* contain such characters, reading from binary files results in byte strings.

In [ ]:
hw1 = b"Hello,\x00\x01\x02world!"
print( hw1 )

Characters from a byte string you can access using indices, just like you do with regular strings. The difference here is that with regular strings you get letters, while with byte strings you get numbers. The numbers are the codes for the letters, which you would also get when you use the `ord()` function on the corresponding letter.

In [ ]:
hw1 = "Hello, world!"
hw2 = b"Hello, world!"

for c in hw1:
    print( c, end=" " )
print()
for c in hw1:
    print( ord( c ), end=" " )
print()
for c in hw2:
    print( c, end=" " )

Since bytes are numbers between 0 and 255, you might want to convert a number to a single-character byte string, or a list of numbers to a multi-character byte string. You can do so using a `bytes` casting on a list of those numbers. Note that if you want to convert a single character, you still have to use a list, but a list with just one element. Do not forget to put the list brackets around that element, because if you do not, the result will not be what you expect.

In [ ]:
bs = bytes( [72, 101, 108, 108, 111, 44, 32, 119, 111, 114, 108, 100, 33] )
print( bs )
bch = bytes( [72] )
print( bch )
wrong = bytes( 72 )
print( wrong )

Can you convert from a byte string to a regular string? You might think that string casting works, but unfortunately it does not:

In [ ]:
hw1 = b"Hello, world!"
hw2 = str( hw1 )
print( hw2 )

The reason that it does not, is that when a string is in the format of a byte string, it uses an encoding scheme, according to the Unicode standard (discussed in a previous chapter). You have to "decode" the byte string according to a certain decoding scheme, which usually is "utf-8", as that is the most common Unicode format. You decode using the `decode()` method, with the encoding scheme as a string parameter. You can also go from a string to a byte string by encoding using the `encode()` method, again with the encoding scheme as string parameter.

In [ ]:
hw1 = b"Hello, world!"
hw2 = hw1.decode( "utf-8" )
print( hw2 )
hw3 = hw2.encode( "utf-8" )
print( hw3 )

In general you have little reason to read text files in binary mode, at least not if you just want to access the text, and so you do not have to worry about encoding and decoding byte strings. The exception is when you have to deal with a text file that uses Unicode characters. Such a file cannot be treated as a text file, and you have to open it in binary mode.

### Binary reading demonstration

To demonstrate how reading a binary file works, I now open the file `pc_rose.txt`, and read ten times ten bytes from it.

In [ ]:
fp = open( "pc_rose.txt", "rb" )
for i in range( 10 ):
    buffer = fp.read( 10 )
    print( buffer )
fp.close()

When you run the code, you see the ten byte strings being displayed. You may also notice that there are certain control characters visible, such as `\r` and `\n`. The `\r` you would not see if you read this file as a text file, because Python converts it, together with the following `\n`, to a single `\n`. Moreover, in a regular string you would not see the `\n`, because it is a newline character which tells Python to move to the next line. 

In the second demonstration, I show what happens if you read an actual binary file, namely a JPG image file. Again, I read the first ten times ten bytes.

In [ ]:
fp = open( "pc_foot.jpg", "rb" )
for i in range( 10 ):
    buffer = fp.read( 10 )
    print( buffer )
fp.close()

You will not be able to make much of the contents of this file, except maybe that you recognize the letters `JFIF` on the first line, which indicate the encoding scheme used for this file. For the rest, it is all rubbish.

---

## Writing a binary file

You write to a binary file using the `write()` methods. The difference with writing to text files is that you have to supply a byte string as argument, rather than a regular string. The following code creates a binary file with some text in it.

In [ ]:
from os.path import getsize

FILENAME = "pc_binarytest.tmp"

fp = open( FILENAME, "wb" )
fp.write( b"And now for something completely different...\x0A\x00\x00\x00\x00\
\xD4\xE8\xE5\xA0\xD3\xF0\xE1\xEE\xE9\xF3\xE8\xA0\xC9\xEE\xF1\xF5\xE9\xF3\xE9\xF4\xE9\xEF\xEE\x00\x00\x00")
fp.close()

print( getsize( FILENAME ), "bytes written" )

Run the code above to create the binary file. The code below opens it in text mode (you can do that, as Python cannot know that it actually is a binary file), reads the contents, and prints the contents. You will see some readable text and some unreadable characters.

In [ ]:
FILENAME = "pc_binarytest.tmp"

fp = open( FILENAME )
while True:
    buffer = fp.readline()
    if buffer == "":
        break
    print( buffer )
fp.close()

**Exercise**: Change the code above to open the file in binary mode and print the contents.

---

## Positioning the file pointer

The file "pc_binarytest.tmp" actually contains a few secret words, which you cannot recognize when printing the file. I am going to use them as an illustration on how to move the file pointer.

The file pointer indicates where in the file you start reading or writing. You can move the file pointer with the `seek()` method. `seek()` gets two integer arguments, of which the second one is optional. The first argument is a relative byte position. The second is the position relative to which you want to move the file pointer. 

The second argument can be 0, 1, or 2. 0 means "relative to the beginning of the file", 1 means "relative to the current file pointer position", and 2 means "relative to the end of the file". If you do not specify a second argument, it is assumed to be 0. In the `os` module there are constants for this argument: `os.SEEK_SET` is 0, `os.SEEK_CUR` is 1, and `os.SEEK_END` is 2.

The first parameter indicates how many bytes you move from the indicated position. When starting at the beginning of the file, it should be a positive number; when starting at the end of the file, it should be a negative number; when starting somewhere in the middle of the file, it can be positive or negative. For instance, the statement `fp.seek(5)` is equivalent to `fp.seek(5,0)`, which moves the file pointer 5 bytes up from the start of the file, placing it at the sixth byte.

Should you wish to know at which position the file pointer is currently placed, you can use the `tell()` method. Both `seek()` and `tell()` can be called for text files too, but are not very useful then.

Now, the secret words are found starting at position 50, and run for a length of 23 bytes. The encoding is such that if you subtract 128 from byte values, you get the ordinals for the letters. So, here is how you get the words out of the file:

In [ ]:
fp = open( "pc_binarytest.tmp", "rb" )
print( "1. Current position of the file pointer is", fp.tell() )
fp.seek( 50 )
print( "2. Current position of the file pointer is", fp.tell() )
buffer = fp.read( 23 )
print( "3. Current position of the file pointer is", fp.tell() )
fp.close()

print( buffer )
s = ""
for c in buffer:
    s += chr( c-128 )
print( "The secret words are:", s )

The `seek()` method is particularly useful when you open a file in "reading and writing" mode ("`r+b`"). It allows you to move through the file, reading where you need to read, and (over)writing where you need to (over)write.

**Exercise**: In the code block below, open the file "pc_binarytest.tmp" in binary "reading and writing" mode, and overwrite the encoded secret words with their decoded translation. Once you have closed the file, open it again in text mode, read the contents, and display them. If you did it all correctly, you should see two readable lines. Should you mess up the file in some way, you can always recreate it.

In [ ]:
# Overwriting secret words.


---

## What you learned

In this chapter, you learned about:

- Using binary files for reading, writing, and reading plus writing
- Binary `read()` and `write()`
- Byte strings
- Conversion between strings and byte strings using `encode()` and `decode()`
- `seek()` and `tell()` methods

---

## Exercises

### Exercise 18.1

Create a simple file encryption program. Open a file and read it in binary mode. For each byte, if it is smaller than 128, add 128; if it is bigger than or equal to 128, subtract 128. Overwrite the byte with new value. Test the program on a copy of one of the small supplied text files (you can create a copy in the notebooks main folder by selecting it and choosing the "Duplicate" button). Check the contents of the encrypted file: they should be a mess. However, when you run the program again, the original file should be restored. If it isn't, you have a bug in your program. Aren't you glad you were only working on a copy?

In [ ]:
# Encryption.


### Exercise 18.2

The fourteen most common letters in the English language are: `etaoinshrdlcum`. Write a text compression program based on this fact. The compression program stores these letters in half-bytes. A half-byte can take the numbers zero to 15. If we only use the numbers 1 to 15, each number can represent one of these fourteen most common letters, and we can use the number 15 for the space. So we can store two of these letters (or space) in a byte (the value for the whole byte would be 16 times the value for the first letter, plus the second letter). If in the text we encounter a letter that is not amongst these fifteen, we indicate that by storing a zero-byte, followed by a whole byte that represents the unencoded letter. Of course, in this setup it is possible that the full byte is actually divided over two bytes, namely the second half-byte of one byte, and the first half-byte of the other byte.

Hint: an easy approach is to build a list of half-bytes. For the most common letters, you store their index-value for the string "`etaoinshrdlcum`&nbsp;&nbsp;" plus 1 (which is a value in the range 1 to 15; notice that the last character in the string is the space). For the other characters, you store three half bytes, namely zero, followed by the ordinal value of the character divided by 16 (rounded down), followed by that ordinal value modulo 16. Once the half-byte-list is finished, you can turn it into a byte list by taking pairs of half-bytes, multiplying the first by 16 and adding the second. That list you can convert to a byte string using `bytes()` casting. 

For testing: the string `Hello, world!`, which is 13 characters in length, will become the 11-character byte string `b'\x04\x81\xbb@,\xf0wI\xba\x02\x10'` if you follow the procedure outlined above (which assigns `e` the value 1, `t` the value 2, etcetera).

In [ ]:
# String compression.


A note on the translation of `Hello, world!` to the given byte string: You may remember that a hexadecimal representation of a byte consists of two hexadecimal digits, i.e., each digit is a half-byte. Using that information, you can see how the translation has been done. The first byte is `\x04`, i.e., the first half-byte is zero. That means that the first character is given literally, i.e., it consists of the second half-byte of `\x04`, and the first half-byte of the next byte, which is `\x81`. That is the byte `\x48`. If you look up the hexadecimal code `48` in the ASCII table (given in the chapter on strings), you see that that is the character `H`. The following half-byte is the second half-byte of `\x81`, i.e., it is `1`. Since this is not zero, it is one of the most common characters, namely the first one, which is `e`. So now you see how `Hello, world!` is compressed as the byte string provided. The byte string does contain a few characters that are not displayed as their hexadecimal code; if you really want to know which hexadecimal code they represent, look them up in the ASCII table.

### Exercise 18.3

As a collary to the previous exercise, write a decompression program for the produced strings.

Hint: Just do the opposite of what you did in the previous exercise: rebuild the half-byte-list. That list is then easily converted back to the original string.

In [ ]:
# String decompression.


### Exercise 18.4

This chapter is about binary files, and the previous two exercises were not, at least, not directly. There simply is not much that you can exercise with where binary files are concerned; the main problems are with handling byte values, which is what the previous two exercises were concerned with. But to round off what these two exercises did, let's now use what you developed in them to compress files.

Write a program that asks for an input file, that must exist, and an output file, that should not exist. Then it asks whether you want to compress or decompress. If you choose compress, the input file is compressed using the method developed above, and written as the output file. If you choose decompress, the input file is decompressed under the assumption that it was compressed with the method developed above, and written as the output file. So you should be able to get the original file again by first compressing and then decompressing.

You best read the whole file in memory before (de)compressing, so that you do not get into problems when a byte string ends in half a byte instead of a full byte after compression. You also best treat both the input file and the output file as binary files. 

Hint: When you compress `pc_amontillado.txt`, 13246 bytes are read,and 9556 bytes are written.

In [ ]:
# File compression/decompression.


---

## Python 2

Python 2 does not have byte strings. There is no need for them in Python 2, as it does not support Unicode either. In Python 2, if you want to write a byte with ordinal value zero, you just write `chr(0)`. The `read()` and `write()` methods for binary files use regular strings in Python 2. This is not allowed in Python 3.

---

End of Chapter 18. Version 1.1. 